In [245]:
### First we handle our files ###
tree_path = f"../000_000_341.tree"
alignment_path = f"../000_000_341.fas.alg"
table_path = f"../000_000_341.tsv"
uniprot_path = f"../uniprot_2018_09.json"

### And we store them in different variables ###
try:
    with open(table_path, "r") as table_file:
        table_info = table_file.readlines()
except:
    print ("No table path.")
        
try:
    with open(alignment_path, "r") as alignment_file:
        alignment_info = alignment_file.read()
except:     
    print ("No alignment path.")
        
import json
uniprot_info = {}
with open(uniprot_path, "r") as uniprot_file:
    for line in uniprot_file:
        uniprot_entry = json.loads(line)
        uniprot_info.update(uniprot_entry)

In [246]:
### LOADING THE SPECIFIED TAG INFORMATION FROM OUR ALIGNMENT ###
def retrieve_features (feature_tag, table_file):
    uniprot_hit_hash = {}
    for line in table_file:
        hit_type = line.split("\t")[2]
        hit_name = line.split("\t")[1]
        if (hit_type == ("swissprot_best" or "swissprot_exact") and hit_name not in uniprot_hit_hash):
            features_newlist = []
            for feature in uniprot_info[hit_name]["FT"]:
                if feature["ft"] == feature_tag:
                    features_newlist.append(feature)
            if len(features_newlist) > 0:
                uniprot_hit_hash[hit_name] = features_newlist
    return uniprot_hit_hash

In [247]:
uniprot_hit_hash = retrieve_features("ACT_SITE", table_info)
print (uniprot_hit_hash)

{'O67077': [{'ft': 'ACT_SITE', 's': '419', 'e': '419', 'ann': '{ECO:0000255|HAMAP-Rule:MF_01458}.'}]}


In [248]:
### LOADING THE TREE WITH ETE3 ###    
from ete3 import PhyloTree, TreeStyle
tree = PhyloTree(tree_infile, alignment=alignment_info, alg_format="fasta")
# tree.show()
# tree.render('%%inline')

In [249]:
### TRANSFORMING THE SEQUENCE POSITIONS INTO ALIGNMENTS POSITIONS ###
def get_alignment_position (sequence_position, sequence):
    alignment_position = 0
    aminoacid_counted = 0
    for aminoacid in sequence:
        if aminoacid.isalpha():
            aminoacid_counted += 1
        if not (aminoacid_counted <= sequence_position or aminoacid_counted == 0):
            break
        alignment_position += 1
    return alignment_position

In [250]:
### GETTING A POSITIONS MATRIX FROM THE FEATURES IN OUR TREE ###
def get_positions_matrix (feature_hash, tree):
    position_matrix = []
    for unigene in feature_hash:
        unigene_sequence = (tree&unigene).sequence
        for feature in feature_hash[unigene]:
            feature_start = int(feature["s"])-1
            feature_end = int(feature["e"])-1
            alignment_feature_start = get_alignment_position(feature_start, unigene_sequence)
            alignment_feature_end = get_alignment_position(feature_end, unigene_sequence)
            for position in range (alignment_feature_start, alignment_feature_end+1):
                position_matrix.append(position)
    position_matrix = sorted(list(set(position_matrix)))
    return position_matrix

position_matrix = get_positions_matrix(uniprot_hit_hash, tree)
print (position_matrix)

[559]


In [251]:
from ete3 import SeqMotifFace
for leaf in tree.iter_leaves():
# for leaf in tree.get_leaves():
    for position in position_matrix:
        seqFace = SeqMotifFace(leaf.sequence[position], seq_format="seq")
        (tree&leaf.name).add_face(seqFace, 0, "aligned")
tree.render('%%inline')
tree.show()